# Pre-Processing Water Level Measurements

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

INPUT_DATE_FORMAT = "%d.%m.%Y %H:%M:%S" 
OUTPUT_DATE_FORMAT = "%Y_%m_%d_%H_%M_%S_%f"

Parse raw measurements file and map levels to "percentage full".

In [ ]:
input_file_water_level_measurements = "DEFINE PATH TO MEASUREMENTS"
output_file_water_level_measurements = "DEFINE OUTPUT PATH"

Get sensor information:

In [ ]:
with open(input_file_water_level_measurements, "r") as sensor_information:
    
    lines = sensor_information.readlines()
    
    print(lines[5])
    print(lines[2])
    print(lines[24])
    
    print(lines[12])
    print(lines[13])

Read file and ignore unused columns:

In [ ]:
input_date_parser = lambda date_string: datetime.strptime(date_string, INPUT_DATE_FORMAT)

water_level = pd.read_csv(input_file_water_level_measurements, 
                          skiprows=47, 
                          skipfooter=2,
                          delimiter="\t",
                          names=["Counter",
                                 "Unfiltered [mm]",
                                 "Filtered [mm]",
                                 "Temperature [°C]",
                                 "Quality",
                                 "Warm Up",
                                 "Date/time",
                                 "Timestamp",
                                 "Raw Data"],
                          parse_dates=["Date/time"],
                          date_parser=input_date_parser)


In [ ]:
filtered_column = "Filtered [mm]"

## Evaluation of difference Smoothed Water Level - Filtered

In [ ]:
filtered_diff = water_level[filtered_column].diff()

quantile_25 = filtered_diff.quantile(.25)
quantile_75 = filtered_diff.quantile(.75)

iqr = quantile_75 - quantile_25

lower_innter_fence = quantile_25 - 1.5 * iqr
lower_outer_fence = quantile_25 - 3 * iqr

upper_inner_fence = quantile_75 + 1.5 * iqr
upper_outer_fence = quantile_75 + 3 * iqr

In [ ]:
figure, axes = plt.subplots(nrows=3, ncols=2, sharex=True, figsize=(15,15))
figure.tight_layout()

water_level["Unfiltered [mm]"].plot(ax=axes[0,0], title="Unfiltered[mm]")
water_level["Unfiltered [mm]"].diff().plot(ax=axes[0,1], title="Unfiltered Change[mm]")

water_level[filtered_column].plot(ax=axes[1,0], title="Filtered[mm]")
filtered_diff.plot(ax=axes[1,1], title="Filtered Change[mm]")

difference = water_level["Unfiltered [mm]"] - water_level[filtered_column]
difference.plot(ax=axes[2,0], title="Difference")

axes[1, 1].axhline(lower_innter_fence, color="green")
axes[1, 1].axhline(lower_outer_fence, color="red")

axes[1, 1].axhline(upper_inner_fence, color="green")
axes[1, 1].axhline(upper_outer_fence, color="red")

plt.show()
plt.close()

Use the interquartile range to define and remove outliers:

In [ ]:
remove_mild_outliers = False
remove_extreme_outliers = True

if remove_extreme_outliers:
    lower_extreme_outliers = filtered_diff < lower_outer_fence
    upper_extreme_outliers = filtered_diff > upper_outer_fence
 
    water_level.loc[lower_extreme_outliers, filtered_column] = np.nan
    water_level.loc[upper_extreme_outliers, filtered_column] = np.nan

if remove_mild_outliers:
    lower_mild_outliers = filtered_diff < lower_innter_fence
    upper_mild_outliers = filtered_diff > upper_inner_fence
    
    water_level.loc[lower_mild_outliers, filtered_column] = np.nan
    water_level.loc[upper_mild_outliers, filtered_column] = np.nan

In [ ]:
SAMPLE_WINDOW_SIZE = 5
smooth_water_level = water_level[filtered_column].rolling(SAMPLE_WINDOW_SIZE).mean()
smooth_water_level = smooth_water_level.to_frame()
idx = smooth_water_level[smooth_water_level.isna().any(axis=1)].index


n = len(idx)

current_length = 0
max_length = 0
end = 0



for i in range(1, n):
    if idx[i-1] + 1 == idx[i]:
        current_length += 1
    else:
        if max_length < current_length:
            max_length = current_length
            end = idx[i-1]
        
        current_length = 0

print(max_length)
print(end)
print("")
for i in idx:
    print(i)
    
    

In [ ]:
SAMPLE_WINDOW_SIZE = 5
smooth_water_level = water_level[filtered_column].rolling(SAMPLE_WINDOW_SIZE).mean()
smooth_water_level = smooth_water_level.interpolate()

In [ ]:
figure, axes = plt.subplots(nrows=3, ncols=2, sharex=True, figsize=(15,15))
figure.tight_layout()

water_level[filtered_column].plot(ax=axes[0,0], title="Filtered[mm]")
water_level[filtered_column].diff().plot(ax=axes[0,1], title="Filtered Change[mm]")

axes[0,1].axhline(lower_innter_fence, color="green")
axes[0,1].axhline(lower_outer_fence, color="red")

axes[0,1].axhline(upper_inner_fence, color="green")
axes[0,1].axhline(upper_outer_fence, color="red")

smooth_change = smooth_water_level.diff()

smooth_water_level.plot(ax=axes[1,0], title="Smoothed Filtered Water Level[mm]")
smooth_change.plot(ax=axes[1,1], title="Smoothed Change[mm]")

smooth_diff = smooth_water_level - water_level[filtered_column]
smooth_diff.plot(ax=axes[2,1], title="Difference Smoothed - Filtered[mm]")

plt.show()
plt.close()


Convert measurements to "percentage full".

In [ ]:
dmax = 190.1 # empty pipe
dmin = 98  # full pipe

pipe_height = dmax - dmin

percentage_full = (dmax - smooth_water_level) / pipe_height

print("Mean percentage full: ", percentage_full.mean())
print("Min percentage full: ", percentage_full.min())
print("Max percentage full: ", percentage_full.max())

print("# Items < 0: ", percentage_full[percentage_full < 0].count())
print("# Items > 1: ", percentage_full[percentage_full > 1].count())

print("\nAfter clipping values:")

percentage_full = percentage_full.clip(upper=1.0, lower=0)
print("Mean percentage full: ", percentage_full.mean())
print("Min percentage full: ", percentage_full.min())
print("Max percentage full: ", percentage_full.max())

In [ ]:
percentage_full.plot()

In [ ]:
percentage_full

In [ ]:
sorted_percentage = percentage_full.to_frame()
sorted_percentage = sorted_percentage.sort_values(by="Filtered [mm]", ignore_index=True)



figure, axes = plt.subplots(nrows=1, ncols=1, sharex=True, figsize=(15,10))
figure.tight_layout()

sorted_percentage.plot(ax=axes)

axes.axhline(0.76, color="red")
axes.axhline(0.75, color="green")
axes.axhline(0.745)
axes.axhline(0.57)
axes.axhline(0.575)
axes.axhline(0.625)


Combine data with timestamps:

In [ ]:
final_data = pd.DataFrame({
    "Timestamp": water_level["Date/time"], 
    "Waterlevel [mm]": smooth_water_level,
    "Percentage Full [%%]": percentage_full
})

final_data.set_index("Timestamp", inplace=True)
final_data = final_data[SAMPLE_WINDOW_SIZE-1:]

final_data.to_csv(output_file_water_level_measurements, date_format=OUTPUT_DATE_FORMAT)

In [ ]:
pd.cut(smooth_water_level, 3, retbins=True)

In [ ]:
smooth_water_level.quantile(.93)

In [ ]:
percentage_full#["Filtered [mm]"]

In [ ]:
percentage_full[percentage_full > .8]
#percentage_full[!(.7 <= percentage_full < .9)].plot()

In [ ]:
percentage_full[(percentage_full <= .4) | (percentage_full > 0.8)].plot()

In [ ]:
percentage_full[((percentage_full <= .4) | (percentage_full > 0.8))].plot()